<a href="https://colab.research.google.com/github/jrg94/CSE5522/blob/lab1/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.0**: Set up the environment (you can click on the play button below to import the appropriate modules).

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**1.1** Read the data from GitHub into a pandas dataframe. 

In [0]:
micro_rna_data_url = 'https://raw.githubusercontent.com/efosler/cse5522data/master/miRScore-x.csv'
micro_ran_dataframe = pd.read_csv(micro_rna_data_url)
micro_rna_well_data_url = 'https://raw.githubusercontent.com/efosler/cse5522data/master/mRNA-y-well-explained.csv'
micro_rna_well_dataframe = pd.read_csv(micro_rna_well_data_url)

**1.2** Print out the top of the dataframe to make sure that the data loaded correctly.  It should be a data table with two columns (price, sf), and 5 rows, with the first row 220000,2240.

In [6]:
micro_ran_dataframe.head()

,Unnamed: 0,MIRC1,MIRC2,MIRC3,MIRC4,MIRC5,MIRC6,MIRC7,MIRC8,MIRC9,MIRC10,MIRC11,MIRC12,MIRC13,MIRC14,MIRC15,MIRC16,MIRC17,MIRC18,MIRC19,MIRC20,MIRC21
0,1,5.453767,4.383828,3.216983,3.428224,7.308519,5.509137,5.515894,3.371819,6.036913,2.876100,8.064057,3.388470,3.931960,6.401138,6.571827,3.785073,5.201207,2.575824,3.977328,5.501027,4.756962
1,2,4.321988,3.986583,3.829885,4.502739,1.685509,5.206405,6.445237,3.465323,6.618933,2.851471,7.730359,2.738890,3.567882,4.360902,6.082557,3.343985,6.303041,3.140004,4.199990,5.224254,4.429511
2,3,4.089116,4.178590,3.393789,3.582799,8.103207,4.997112,5.359251,3.681484,6.373393,2.851890,8.934644,2.892689,3.132659,6.555559,6.559177,3.469511,5.373332,4.424949,3.138479,5.682007,4.403253
3,4,4.884692,4.061140,3.475111,4.017083,8.005837,5.491073,5.014364,4.300977,6.017040,2.834699,8.646215,3.163199,3.531044,6.908134,6.624755,3.833247,5.495557,4.954630,4.542415,5.469924,4.693171
4,5,4.053976,4.224208,3.801616,3.329253,9.208897,5.518924,5.534329,2.391842,6.183707,3.029387,7.351146,2.873874,3.305899,6.917566,6.138030,3.250404,5.502236,4.572714,2.677879,5.577007,4.290140
